In [0]:
dbutils.widgets.text("Start_Date","2024-07-25")
start_date = dbutils.widgets.get("Start_Date")

In [0]:
customer_data=f"/Volumes/workspace/default/scd/Customers/customer_{start_date}.csv"
print(customer_data)
df2= spark.read.csv(customer_data,header=True,inferSchema=True,multiLine=True)

In [0]:
from pyspark.sql.functions import *
df2 = df2.withColumn("Start_Date",lit(start_date))
df2 = df2.withColumn("End_Date",lit("9999-12-31"))
df2 = df2.withColumn("Current_Flag",lit("Y"))

In [0]:
df2.printSchema()

In [0]:
df2 = df2.withColumn("phone_number", col("phone_number").cast("string"))

In [0]:
display(df2)

In [0]:
# df2.write \
#     .format("delta") \
#     .mode("overwrite") \
#     .saveAsTable("default.base_customers1")

In [0]:
from delta.tables import DeltaTable
delta_table = DeltaTable.forName(spark, "default.base_customers1")
delta_table.alias("target").merge(
    source=df2.select("customer_id", "start_date").alias("source"),
    condition="target.customer_id = source.customer_id AND target.current_flag = 'Y'"
).whenMatchedUpdate(set={
    "end_date": "source.start_date",
    "current_flag": "'N'"
}).execute()

In [0]:
df2.write \
    .format("delta") \
    .mode("append") \
    .saveAsTable("default.base_customers1")

In [0]:
updated_customers = spark.table("default.base_customers1")
updated_customers.display()